In [1]:
import os
os.chdir('/nfs/homedirs/rachwan/Torch-Pruning/examples/yolo_example/yolov5')
import torch
import numpy as np

In [2]:
model = torch.load('/nfs/homedirs/rachwan/Torch-Pruning/examples/yolo_example/yolov5/runs/train/exp283/weights/best.pt')['model'].cuda()
# model = checkpoint['model'].cuda()
# from models.common import DetectMultiBackend
# model = DetectMultiBackend('/nfs/homedirs/rachwan/Torch-Pruning/examples/yolo_example/yolov5/runs/train/exp289/weights/best.pt', device='cuda', dnn=False, data='coco128.yaml', fp16=False)

model.float()
model.train()
for k, v in model.named_parameters():
    v.requires_grad = True  # train all layers

In [3]:
params = sum([np.prod(p.size()) for p in model.parameters()])
print("Number of Parameters: %.1fM" % (params / 1e6))

Number of Parameters: 45.2M


In [4]:
torch.cuda.memory_allocated()

186416640

In [5]:
torch.cuda.max_memory_allocated()

188413440

In [6]:
torch.cuda.reset_max_memory_allocated()

/nfs/homedirs/rachwan/miniconda3/envs/structure_pruning/lib/python3.10/site-packages/torch/cuda/memory.py:278: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


In [7]:
torch.cuda.max_memory_allocated()

186416640

In [8]:
from evaluation_tool.models.metrics.Evaluator import Evaluator
evaluator = Evaluator()

In [9]:
from utils.loss import ComputeLoss
compute_loss = ComputeLoss(model)

In [10]:
evaluator.evaluate_training(model, tuple([3, 640, 640]), compute_loss, 20, 'cuda', iterations=100)

In [11]:
evaluator.get_all_metrics()

{'time': 12.1259716796875,
 'gpu': 27.722969088000003,
 'disk': 1535.6424379999999,
 'cost': 5558.849999999999,
 'emission': 0.016660547701994377,
 'energy': 0.055350656817257066}

In [1]:
import os
os.chdir('/nfs/homedirs/rachwan/Torch-Pruning/examples/yolo_example/yolov5')
import torch
import numpy as np
import yaml

In [2]:
from models.yolo import Model
from utils.torch_utils import de_parallel

In [3]:
model = Model('models/yolov5x.yaml', ch=3, nc=80, anchors=3).to('cuda')  # create
# checkpoint = torch.load('/nfs/homedirs/rachwan/Torch-Pruning/yolov5n.pt')
# model = checkpoint['model'].cuda()
# model = torch.hub.load('ultralytics/yolov5', 'yolov5x', force_reload=True)
model.float()
model.train()
for k, v in model.named_parameters():
    v.requires_grad = True  # train all layers
model.hyp = {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0006328125000000001, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'warmup_bias_lr': 0.1, 'box': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0, 'label_smoothing': 0.0}
model(torch.rand([20] + list([3,640,640])).to('cuda'))
params = sum([np.prod(p.size()) for p in model.parameters()])
print("Number of Parameters: %.1fM" % (params / 1e6))

Overriding model.yaml anchors with anchors=3

                 from  n    params  module                                  arguments                     
  0                -1  1      8800  models.common.Conv                      [3, 80, 6, 2, 2]              
  1                -1  1    115520  models.common.Conv                      [80, 160, 3, 2]               
  2                -1  4    309120  models.common.C3                        [160, 160, 4]                 
  3                -1  1    461440  models.common.Conv                      [160, 320, 3, 2]              
  4                -1  8   2259200  models.common.C3                        [320, 320, 8]                 
  5                -1  1   1844480  models.common.Conv                      [320, 640, 3, 2]              
  6                -1 12  13125120  models.common.C3                        [640, 640, 12]                
  7                -1  1   7375360  models.common.Conv                      [640, 1280, 3, 2]     

Number of Parameters: 86.7M


In [4]:
from evaluation_tool.models.metrics.Evaluator import Evaluator
evaluator = Evaluator()
from utils.loss import ComputeLoss
compute_loss = ComputeLoss(model)
evaluator.evaluate_training(model, tuple([3, 640, 640]), compute_loss, 20, 'cuda', iterations=100)
evaluator.get_all_metrics()

torch.Size([20, 3, 80, 80, 85])
torch.Size([222, 6])


{'time': 14.0243359375,
 'gpu': 28.51089152,
 'disk': 2948.0350639999997,
 'cost': 6428.4,
 'emission': 0.019931808907760425,
 'energy': 0.06621863424505126}